In [1]:
import torch
import pandas as pd
import numpy as np
from torch import nn, optim
import torch.utils.data as data_utils
from torch.utils.data import TensorDataset, DataLoader  # Dataset, DataLoader
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt   # 도식화
import tqdm   # 진행률 표시

In [3]:
## 자르고 합치기
TrainDataSet = pd.read_csv("/opt/ml/segmentation/fbi/merge(no_kor_실내활동_no_id).csv")

## NAN 결측치 제거하기
TrainDataSet = TrainDataSet.fillna(0)
y_train = TrainDataSet[['실내활동']]
TrainDataSet.drop(columns = '실내활동', axis=1,inplace=True)

## 학습 데이터 => numpy로 변경
x_train = TrainDataSet.loc[:]

In [4]:
x_train.loc[(x_train.price <= 2500), 'price'] = 0
x_train.loc[(x_train.price > 2500) & (x_train.price <= 3500), 'price'] = 1
x_train.loc[(x_train.price > 3500) & (x_train.price <= 5000), 'price'] = 2
x_train.loc[(x_train.price > 5000), 'price'] = 3

In [5]:
print(x_train)

      type  top_sheet_ingredient  absorbent_core  made_in  price
0       12                    27              30       31      0
1       12                    27              30       31      0
2       12                    26              29       31      0
3       12                    26              29       31      0
4       12                    26              29       31      1
...    ...                   ...             ...      ...    ...
4780    12                    25              28       32      1
4781    12                    25              28       32      2
4782    12                    25              28       31      1
4783    12                    25              28       31      2
4784    12                    25              28       32      3

[4785 rows x 5 columns]


In [6]:
# y값 바꿔주기
# y_train = y_train['냄새'].replace(['아주 약함', '약함', '보통(냄)', '강함', '아주 강함'],[0, 1, 2, 3, 4])
# y_train = y_train['생리량'].replace(['아주 적음', '적음', '보통', '많음', '아주 많음'],[0, 1, 2, 3, 4])
# y_train = y_train['생리통'].replace(['통증 없음', '조금 아픔', '아픔', '많이 아픔', '매우 아픔'],[0, 1, 2, 3, 4])
# y_train = y_train['신체활동'].replace(['달리기', '자전거 타기', '헬스', '댄스', '요가'],[0, 1, 2, 3, 4])
y_train = y_train['실내활동'].replace(['TV보기', '동영상 시청', '독서', '아이돌보기', '친구와 통화하기', '공부하기'],[0, 1, 2, 3, 4, 5])

# output값 직접 지정하기
x_train = np.array(x_train, dtype= np.float64)
y_train = np.array(y_train, dtype= np.int64)


x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)
y_train = y_train.reshape(-1) ########

In [7]:
ds_train = TensorDataset(x_train, y_train)

train_loader = DataLoader(ds_train, batch_size=128, shuffle=True)


In [8]:
k = 30
net = nn.Sequential(
    nn.Linear(5,k),
    nn.ReLU(),
    nn.BatchNorm1d(k),
    nn.Linear(k,k),
    nn.ReLU(),
    nn.BatchNorm1d(k),
    nn.Linear(k,k),
    nn.ReLU(),
    nn.BatchNorm1d(k),
    nn.Linear(k,6),
)


In [9]:
# 훈련용 헬퍼 함수
def train_net(net, train_loader,
              optimizer_cls=optim.Adam,
              loss_fn=nn.CrossEntropyLoss(),learning_rate=0.05,
              n_iter=100, device="cpu"):
    train_losses = []
    train_acc = []
    val_acc = []
    optimizer = optimizer_cls(net.parameters(),lr = learning_rate)
    for epoch in range(n_iter):
        running_loss = 0.0
        # 신경망을 훈련 모드로 설정
        net.train()
        n = 0
        n_acc = 0
        # 시간이 많이 걸리므로 tqdm을 사용해서 진행바를 표시
        for i, (xx, yy) in tqdm.tqdm(enumerate(train_loader),
            total=len(train_loader)):
            xx = xx.to(device)
            yy = yy.to(device)
            h = net(xx)
            loss = loss_fn(h, (yy-min(y_train)))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            n += len(xx)
            _, y_pred = h.max(1)
            n_acc += (yy == (y_pred+min(y_train))).float().sum().item()
        train_losses.append(running_loss / i)
        # 훈련 데이터의 예측 정확도
        train_acc.append(n_acc / n)
        
        # print("softmax : ",torch.nn.functional.softmax(h, dim=1))        
        print("softmax : ",torch.nn.functional.softmax(h, dim=1)[0] * 100)




In [10]:

# 신경망의 모든 파라미터를 GPU로 전송
net.to("cuda")

# 훈련 실행
train_net(net, train_loader, n_iter=100,optimizer_cls = optim.SGD , learning_rate = 0.05, device="cuda")
torch.save(net.state_dict(), "./submission/pretrained_실내활동.pt")

  3%|▎         | 1/38 [00:00<00:03,  9.72it/s]

softmax :  tensor([55.6433, 10.9714,  7.8685,  6.8638,  8.5635, 10.0894], device='cuda:0',
       grad_fn=<MulBackward0>)


  3%|▎         | 1/38 [00:00<00:03,  9.66it/s]

softmax :  tensor([76.1536, 10.7828,  2.1234,  4.4838,  3.1748,  3.2816], device='cuda:0',
       grad_fn=<MulBackward0>)


  3%|▎         | 1/38 [00:00<00:03,  9.72it/s]

...

softmax :  tensor([75.5993, 13.2535,  0.7206,  2.1043,  6.6348,  1.6874], device='cuda:0',
       grad_fn=<MulBackward0>)


  3%|▎         | 1/38 [00:00<00:03,  9.63it/s]

softmax :  tensor([80.2224, 10.8822,  0.5180,  1.5214,  5.3878,  1.4682], device='cuda:0',
       grad_fn=<MulBackward0>)


100%|██████████| 38/38 [00:04<00:00,  9.67it/s]

softmax :  tensor([73.8099, 14.0669,  0.7210,  2.2591,  7.1271,  2.0161], device='cuda:0',
       grad_fn=<MulBackward0>)
